# Modeling an entraining cloud updraft

This notebook calculates the time evolution of four variables:

\[velocity, height, $\theta_{ecld}$, $r_{Tcloud}$ \]

in a rising, entraining cloud with constant entrainment rate.  The environment is specified from a Wyoming sounding,
and interpolated each timestep using [scipy.interpolate.interp1d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) 

The variables are intergrated with respect to time using [scipy.integrate.RK45](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.RK45.html)

In [1]:
import numpy as np
import pandas as pd
from functools import partial
from pprint import pformat
from a405.thermo.constants import constants as c
from a405.thermo.thermlib import find_Tmoist,find_rsat,find_thetaep
from scipy.interpolate import interp1d
from scipy.integrate import RK45
from a405.soundings.wyominglib import write_soundings, read_soundings
import json

import matplotlib.pyplot as plt
from a405.skewT.nudge import nudge

## Find the derivatives wrt time of each of the 4 variables

See [entrain.pdf](https://www.dropbox.com/scl/fi/uj7sq0hcdbcgtxomly4vd/entrain.pdf?rlkey=feaufh1d7lixg5rtdlxj4vdzu&dl=0)  notes.

For the entrainment calculation, we need environmental temperature, dewpoint and pressure at arbitrary heights.  Use [scipy.interpolate.interp1d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) for this

In [2]:
def derivs(t, y, entrain_rate=None, tinterp=None, tdinterp = None, pinterp=None):
    """Function that computes the derivative vector for the ode integrator

    Parameters
    ----------
    
    t: float
       time (s)
    y: vector
       4-vector containing wvel (m/s), height (m), thetae (K), rt (kg/kg)
    entrain_rate: float
                  1/m dm/dt (s-1)
    tinterp: func
                interp1d function for environmental temperature T(z) 
    tdinterp: func
                interp1d function for environmental dewpoint temperature Td(z)
    pinterp: func
                interp1d function for presusure  p(z)

    Returns
    -------

    yp: vector
       4-vector containing time derivatives of wvel (m/s^2), height (m/s), thetae (K/s), rt (kg/kg/s)
    """
    #print(f"inside derivs")
    #print(f"{y=}")
    yp = np.zeros((4,),dtype=float)
    velocity = y[0]
    height = y[1]
    thetae_cloud = y[2]
    rt_cloud = y[3]
    #
    # fill the yp (yprime) vector with the derivatives
    #
    # yp[0] is the acceleration, in this case the buoyancy 
    #
    yp[0] = calcBuoy(height, thetae_cloud, tinterp, tdinterp, pinterp)
    press = pinterp(height)*100. #Pa
    Tdenv = tdinterp(height) + c.Tc #K
    Tenv = tinterp(height) + c.Tc #K
    rtenv = find_rsat(Tdenv, press) #kg/kg
    thetaeEnv = find_thetaep(Tdenv,Tenv, press)
    #
    # yp[1] is the rate of change of height
    #
    yp[1] = velocity
    #
    # yp[2] is the rate of change of thetae_cloud
    #
    yp[2] = entrain_rate*(thetaeEnv - thetae_cloud)
    #
    # yp[3] is the rate of change of rt_cloud
    #
    yp[3] = entrain_rate*(rtenv - rt_cloud)
    #print(f" derivs returning")
    #print(f"{yp=}")
    return yp

## Find the buoyancy from the cloud and environment $\theta_e$ and $r_T$

In [3]:
def calcBuoy(height, thetae0, tinterp, tdinterp, pinterp):
    """function to calculate buoyant acceleration for an ascending saturated parcel
       this version neglects liquid water loading
    
    Parameters
    ----------
    
    height: float
            parcel height (m)
    thetae0: float
            parcel thetae (K)

    tinterp: func
                interp1d function for environmental temperature T(z) 
    tdinterp: func
                interp1d function for environmental dewpoint temperature Td(z)
    pinterp: func
                interp1d function for presusure  p(z)

    Returns
    -------

    buoy: float
          buoyancy (m/s/s)
    """
    #input: height (m), thetae0 (K), plus function handles for
    #T,Td, press soundings
    #output: Bout = buoyant acceleration in m/s^2
    #neglect liquid water loading in the virtual temperature
    
    press=pinterp(height)*100.#%Pa
    Tcloud=find_Tmoist(thetae0,press) #K
    rvcloud=find_rsat(Tcloud,press); #kg/kg
    Tvcloud=Tcloud*(1. + c.eps*rvcloud)
    Tenv=tinterp(height) + c.Tc
    Tdenv=tdinterp(height) + c.Tc
    #print(f"inside buoy {(height,Tenv,Tdenv,press)=}")
    rvenv=find_rsat(Tdenv,press); #kg/kg
    Tvenv=Tenv*(1. + c.eps*rvenv)
    TvDiff=Tvcloud - Tvenv
    buoy = c.g0*(TvDiff/Tvenv)
    return buoy

## Integrator 


Use [scipy.integrate.RK45](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.RK45.html) to integrate our system of 4 odes

In [4]:
def integ_entrain(df_sounding,entrain_rate):
    """integrate an ascending parcel given a constant entrainment rate
       this version hardwired to start parcel at 800 hPa with cloud base
       values of environment at 900 hPa

    Parameters
    ----------

    df_sounding: pandas dataframe 
               : cloumns are temperature, dewpoint, height, press

    entrain_rate: float
                  1/m dm/dt (s-1)

    Returns
    -------

    df_out: dataframe
          dataframe containing wvel (m/s) ,cloud_height (m) , thetae (K), rt (kg/kg) for assending parcel

   interpPress: func
              interp1d function for presusure  p(z) (used for plotting)
    """
    press = df_sounding['pres'].values
    height = df_sounding['hght'].values
    temp = df_sounding['temp'].values
    dewpoint = df_sounding['dwpt'].values
    #
    # 
    #
    envHeight= nudge(height)

    interpTenv = interp1d(envHeight,temp)
    interpTdEnv = interp1d(envHeight,dewpoint)
    interpPress = interp1d(envHeight,press)

    args=dict(entrain_rate = entrain_rate,
              tinterp=interpTenv,
              tdinterp=interpTdEnv,
              pinterp=interpPress)
    #
    # use functools.partial to supply the extra arguments to the derivs function
    # this changes the derivs function signature from
    #
    # derivs(t, y, entrain_rate=None, tinterp=None, tdinterp = None, pinterp=None)
    #
    # to 
    #
    # the_derivs(t,y) 
    #
    # as required by the RK45 integrator
    # 
    #
    the_derivs = partial(derivs,**args)
    #
    # set cloudbase (dewpoint=temperature) at 900 hPa
    #
    p900_level = len(press) - np.searchsorted(press[::-1],900.)
    rtcloud = find_rsat(dewpoint[p900_level] + c.Tc, press[p900_level]*100.)
    thetaeVal=find_thetaep(dewpoint[p900_level] + c.Tc,
                           temp[p900_level] + c.Tc,press[p900_level]*100.)
    #
    # start parcel at 800 hPa by keeping thetae and rtcloud at cloudbase
    # values (i.e. assume parcel has risen adiabatically from 900 to 800 hPa
    #
    p800_level = len(press) - np.searchsorted(press[::-1],800.)
    height_800=height[p800_level]
    winit = 0.5 #initial velocity (m/s)
    yinit = [winit, height_800, thetaeVal, rtcloud]  
    tinit = 0  #seconds
    tfin = 2500  #seconds
    dt = 10 #seconds
    tspan = (tinit, tfin)
    output_times = np.arange(tinit, tfin, dt)
    #
    # want to integrate derivs using dopr15 runge kutta described at
    # http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.ode.html
    #
    init_vals = (yinit, tinit)
    rk45 = RK45(the_derivs, tinit, yinit, t_bound=tfin, max_step=dt)
    yout = []
    while rk45.status == "running":
        #
        # stop if wvel < 0
        #
        if rk45.y[0] < 0:
            break
        press = interpPress(rk45.y[1])*100.
        yvals= [rk45.t,press]
        yvals.extend(rk45.y)
        yout.append(yvals)
        rk45.step()
    #
    # convert the output into a pandas dataframe
    #
    colnames=['time','press','wvel','cloud_height','thetae_cloud','rt_cloud']
    df_out=pd.DataFrame.from_records(yout,columns=colnames)
    df_out = df_out.set_index('time',drop=False)
    return df_out

## Read in a sounding to set the environment

In [18]:
write = False
sounding_dir = 'littlerock'
station = 72340
year = 2012
month = 7
if write:
    values=dict(region='naconf',year=year,month=month,start='0100',stop='3000',station=station)
    write_soundings(values, sounding_dir)
    soundings= read_soundings(sounding_dir)
else:
    soundings= read_soundings(sounding_dir)
type(soundings)

dict

In [6]:
day = 9
hour = 0
the_time=(2012,7,day,hour)
sounding=soundings['sounding_dict'][the_time]

## Do the integration

In [7]:
entrain_rate = 2.e-4  #s^{-1}
df = integ_entrain(sounding,entrain_rate)

/Users/phil/mini310/envs/a405/lib/python3.11/site-packages/a405/thermo/thermlib.py:469: RuntimeWarning: invalid value encountered in log
  s = cp * np.log(T) - c.Rd * np.log(pd) + lv * rv / T - vapor_term


In [8]:
df

,time,press,wvel,cloud_height,thetae_cloud,rt_cloud
time,,,,,,
0.000000,0.000000,79310.000000,0.500000,2134.000000,346.552571,0.014049
0.343993,0.343993,79308.418593,0.500719,2134.172120,346.552654,0.014049
3.783925,3.783925,79292.474311,0.508383,2135.907492,346.553483,0.014049
13.783925,13.783925,79244.568623,0.535688,2141.121537,346.555825,0.014046
23.783925,23.783925,79193.800282,0.570767,2146.647152,346.558063,0.014044
...,...,...,...,...,...,...
703.783925,703.783925,13838.788551,23.989843,14728.909939,345.933539,0.012997
713.783925,713.783925,13350.812421,18.927640,14943.866987,345.971263,0.012971
723.783925,723.783925,12999.611882,13.476301,15106.164206,346.012060,0.012945


## Convert the dataframe to xarray

In [19]:
the_ds = df.to_xarray()
dir(the_ds)

['_HANDLED_TYPES',
 '__abs__',
 '__abstractmethods__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__copy__',
 '__dask_graph__',
 '__dask_keys__',
 '__dask_layers__',
 '__dask_optimize__',
 '__dask_postcompute__',
 '__dask_postpersist__',
 '__dask_scheduler__',
 '__dask_tokenize__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',

## Add units to the variables plus dataset attributes

In [10]:
the_ds = the_ds.set_coords(['press','cloud_height'])
the_ds['press'] = the_ds['press'].assign_attrs(units = 'Pa')
the_ds['cloud_height'] = the_ds['cloud_height'].assign_attrs(units = 'm')
the_ds['wvel'] = the_ds['wvel'].assign_attrs(units = 'm/s')
the_ds['thetae_cloud'] = the_ds['thetae_cloud'].assign_attrs(units = 'K')
the_ds['rt_cloud'] = the_ds['rt_cloud'].assign_attrs(units = 'kg/kg')
the_ds.attrs = {'history': ' written by entraining_plume.ipynb',
                'sounding_dir':sounding_dir,
                'sounding_time':the_time,
                'station':station}

In [11]:
the_ds

<xarray.Dataset>
Dimensions:       (time: 77)
Coordinates:
  * time          (time) float64 0.0 0.344 3.784 13.78 ... 723.8 733.8 743.8
    press         (time) float64 7.931e+04 7.931e+04 ... 1.277e+04 1.267e+04
    cloud_height  (time) float64 2.134e+03 2.134e+03 ... 1.521e+04 1.526e+04
Data variables:
    wvel          (time) float64 0.5 0.5007 0.5084 0.5357 ... 13.48 7.746 1.859
    thetae_cloud  (time) float64 346.6 346.6 346.6 346.6 ... 346.0 346.1 346.1
    rt_cloud      (time) float64 0.01405 0.01405 0.01405 ... 0.01292 0.01289
Attributes:
    history:         written by entraining_plume.ipynb
    sounding_dir:   littlerock
    sounding_time:  (2012, 7, 9, 0)
    station:        72340

In [12]:
the_ds['press']

<xarray.DataArray 'press' (time: 77)>
array([79310.        , 79308.41859301, 79292.47431112, 79244.56862311,
       79193.80028175, 79139.43394974, 79080.6855207 , 79016.71134682,
       78946.59670416, 78869.34334973, 78783.85601593, 78688.92767919,
       78583.22342934, 78465.26275595, 78333.40005729, 78185.8031672 ,
       78020.42968639, 77835.00089749, 77626.97304021, 77392.77201197,
       77129.22405282, 76834.44840894, 76505.57083043, 76150.87992253,
       75756.07388254, 75317.04330851, 74829.32104238, 74288.06842964,
       73691.49174984, 73051.69233709, 72343.23971168, 71559.09029657,
       70696.37346091, 69764.60544951, 68781.50187957, 67723.32675656,
       66588.26026199, 65404.18230418, 64154.28577237, 62824.47304962,
       61417.80776212, 59959.59220596, 58445.7246164 , 56873.41437277,
       55274.64175848, 53624.61881464, 51921.05227754, 50151.72940126,
       48340.0441871 , 46509.41509743, 44646.91332996, 42777.47471245,
       40935.06856309, 39093.16749846, 37273.06412729, 35473.8154306 ,
       33712.85559983, 31991.83366127, 30318.4796622 , 28679.01700318,
       27108.01403393, 25619.47114784, 24161.0827308 , 22757.7668053 ,
       21411.92353796, 20148.94733758, 18975.76234576, 17941.22602421,
       16955.33590543, 16016.62973683, 15177.23913599, 14456.81174615,
       13838.78855051, 13350.81242063, 12999.61188189, 12769.59464264,
       12668.13879057])
Coordinates:
  * time          (time) float64 0.0 0.344 3.784 13.78 ... 723.8 733.8 743.8
    press         (time) float64 7.931e+04 7.931e+04 ... 1.277e+04 1.267e+04
    cloud_height  (time) float64 2.134e+03 2.134e+03 ... 1.521e+04 1.526e+04
Attributes:
    units:    Pa

## write the dataset to netcdf

In [13]:
filename = "littlerock.nc"
the_ds.to_netcdf(filename)

In [14]:
!ncdump -h littlerock.nc

netcdf littlerock {
dimensions:
	time = 77 ;
variables:
	double time(time) ;
		time:_FillValue = NaN ;
	double press(time) ;
		press:_FillValue = NaN ;
		press:units = "Pa" ;
	double wvel(time) ;
		wvel:_FillValue = NaN ;
		wvel:units = "m/s" ;
		wvel:coordinates = "cloud_height press" ;
	double cloud_height(time) ;
		cloud_height:_FillValue = NaN ;
		cloud_height:units = "m" ;
	double thetae_cloud(time) ;
		thetae_cloud:_FillValue = NaN ;
		thetae_cloud:units = "K" ;
		thetae_cloud:coordinates = "cloud_height press" ;
	double rt_cloud(time) ;
		rt_cloud:_FillValue = NaN ;
		rt_cloud:units = "kg/kg" ;
		rt_cloud:coordinates = "cloud_height press" ;

// global attributes:
		:history = " written by entraining_plume.ipynb" ;
		:sounding_dir = "littlerock" ;
		:sounding_time = 2012LL, 7LL, 9LL, 0LL ;
		:station = 72340LL ;
}
